In [13]:
import os
import pandas as pd
import numpy as np
import math
import cmath
import matplotlib.pyplot as plt
import seaborn as sns

# Data

In [14]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 300)

In [15]:
df = pd.read_csv("../data/jetblue_df.csv")

In [16]:
df.head()

,Unnamed: 0,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeEpochSeconds,segmentsDepartureTimeRaw,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,5,79eda9f841e226a1e2121d74211e595c,2022-04-16,2022-04-17,ATL,BOS,VH0AUEL1,PT2H38M,0,False,False,True,217.67,248.60,7,947.0,1650206700,2022-04-17T10:45:00.000-04:00,1650216180,2022-04-17T13:23:00.000-04:00,BOS,ATL,JetBlue Airways,B6,NaN,9480,947,coach
1,29,24c07cf38de8a7a0ed2d7709168042a8,2022-04-16,2022-04-17,ATL,BOS,QH0KUEL1,PT2H44M,0,False,False,True,316.73,355.09,7,947.0,1650233220,2022-04-17T18:07:00.000-04:00,1650243060,2022-04-17T20:51:00.000-04:00,BOS,ATL,JetBlue Airways,B6,NaN,9840,947,coach
2,57,a291a2b5ac19223c3407cfec58d1fc24,2022-04-16,2022-04-17,ATL,CLT,VH0AUEL1,PT8H48M,0,False,False,False,482.79,542.60,7,1675.0,1650206700||1650229500,2022-04-17T10:45:00.000-04:00||2022-04-17T17:0...,1650216180||1650238380,2022-04-17T13:23:00.000-04:00||2022-04-17T19:3...,BOS||CLT,ATL||BOS,JetBlue Airways||JetBlue Airways,B6||B6,||Embraer 190,9480||8880,947||728,coach||coach
3,146,d53bcf9e8b6523fa2de9420d7cb22196,2022-04-16,2022-04-17,ATL,DTW,RH0AUEL1,PT5H8M,0,False,False,False,269.78,313.61,7,1247.0,1650235800||1650246600,2022-04-17T18:50:00.000-04:00||2022-04-17T21:5...,1650244380||1650254280,2022-04-17T21:13:00.000-04:00||2022-04-17T23:5...,JFK||DTW,ATL||JFK,JetBlue Airways||JetBlue Airways,B6||B6,Embraer 190||Embraer 190,8580||7680,762||485,coach||coach
4,209,5bb138f39804320ece767e747608a50e,2022-04-16,2022-04-17,ATL,EWR,VC0QBEL1,PT2H9M,0,False,False,True,171.16,198.60,7,762.0,1650195000,2022-04-17T07:30:00.000-04:00,1650202740,2022-04-17T09:39:00.000-04:00,EWR,ATL,JetBlue Airways,B6,Embraer 190,7740,762,coach


In [17]:
sorted(df.columns)

['Unnamed: 0',
 'baseFare',
 'destinationAirport',
 'elapsedDays',
 'fareBasisCode',
 'flightDate',
 'isBasicEconomy',
 'isNonStop',
 'isRefundable',
 'legId',
 'searchDate',
 'seatsRemaining',
 'segmentsAirlineCode',
 'segmentsAirlineName',
 'segmentsArrivalAirportCode',
 'segmentsArrivalTimeEpochSeconds',
 'segmentsArrivalTimeRaw',
 'segmentsCabinCode',
 'segmentsDepartureAirportCode',
 'segmentsDepartureTimeEpochSeconds',
 'segmentsDepartureTimeRaw',
 'segmentsDistance',
 'segmentsDurationInSeconds',
 'segmentsEquipmentDescription',
 'startingAirport',
 'totalFare',
 'totalTravelDistance',
 'travelDuration']

In [18]:
df.shape

(6824440, 28)

In [19]:
df.size

191084320

First need to address that the Unamed:0 column and save that as true. 

In [20]:
df.drop("Unnamed: 0", axis=1, inplace=True)

In [21]:
df.head(100)

columns_to_see_classification_error_in = ["segmentsEquipmentDescription", "segmentsAirlineCode", "segmentsAirlineName", "segmentsArrivalAirportCode", "segmentsArrivalTimeEpochSeconds", "segmentsDepartureTimeEpochSeconds", "segmentsCabinCode"]

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeEpochSeconds,segmentsDepartureTimeRaw,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode
0,79eda9f841e226a1e2121d74211e595c,2022-04-16,2022-04-17,ATL,BOS,VH0AUEL1,PT2H38M,0,False,False,True,217.67,248.60,7,947.0,1650206700,2022-04-17T10:45:00.000-04:00,1650216180,2022-04-17T13:23:00.000-04:00,BOS,ATL,JetBlue Airways,B6,NaN,9480,947,coach
1,24c07cf38de8a7a0ed2d7709168042a8,2022-04-16,2022-04-17,ATL,BOS,QH0KUEL1,PT2H44M,0,False,False,True,316.73,355.09,7,947.0,1650233220,2022-04-17T18:07:00.000-04:00,1650243060,2022-04-17T20:51:00.000-04:00,BOS,ATL,JetBlue Airways,B6,NaN,9840,947,coach
2,a291a2b5ac19223c3407cfec58d1fc24,2022-04-16,2022-04-17,ATL,CLT,VH0AUEL1,PT8H48M,0,False,False,False,482.79,542.60,7,1675.0,1650206700||1650229500,2022-04-17T10:45:00.000-04:00||2022-04-17T17:0...,1650216180||1650238380,2022-04-17T13:23:00.000-04:00||2022-04-17T19:3...,BOS||CLT,ATL||BOS,JetBlue Airways||JetBlue Airways,B6||B6,||Embraer 190,9480||8880,947||728,coach||coach
3,d53bcf9e8b6523fa2de9420d7cb22196,2022-04-16,2022-04-17,ATL,DTW,RH0AUEL1,PT5H8M,0,False,False,False,269.78,313.61,7,1247.0,1650235800||1650246600,2022-04-17T18:50:00.000-04:00||2022-04-17T21:5...,1650244380||1650254280,2022-04-17T21:13:00.000-04:00||2022-04-17T23:5...,JFK||DTW,ATL||JFK,JetBlue Airways||JetBlue Airways,B6||B6,Embraer 190||Embraer 190,8580||7680,762||485,coach||coach
4,5bb138f39804320ece767e747608a50e,2022-04-16,2022-04-17,ATL,EWR,VC0QBEL1,PT2H9M,0,False,False,True,171.16,198.60,7,762.0,1650195000,2022-04-17T07:30:00.000-04:00,1650202740,2022-04-17T09:39:00.000-04:00,EWR,ATL,JetBlue Airways,B6,Embraer 190,7740,762,coach
5,8201f75d221c7aa5b78f3b7781166032,2022-04-16,2022-04-17,ATL,JFK,RC0ABEL1,PT2H15M,0,False,False,True,171.16,198.60,7,762.0,1650200400,2022-04-17T09:00:00.000-04:00,1650208500,2022-04-17T11:15:00.000-04:00,JFK,ATL,JetBlue Airways,B6,Airbus A320,8100,762,coach
6,a63591f2b4df39d809c4a45a3898e122,2022-04-16,2022-04-17,ATL,JFK,RC0ABEL1,PT2H23M,0,False,False,True,171.16,198.60,7,762.0,1650235800,2022-04-17T18:50:00.000-04:00,1650244380,2022-04-17T21:13:00.000-04:00,JFK,ATL,JetBlue Airways,B6,Embraer 190,8580,762,coach
7,79352689384430e23285d3b851fb641e,2022-04-16,2022-04-17,ATL,LGA,B0AJZSN1,PT2H16M,0,False,False,True,171.16,198.60,2,762.0,1650211380,2022-04-17T12:03:00.000-04:00,1650219540,2022-04-17T14:19:00.000-04:00,LGA,ATL,JetBlue Airways,B6,Embraer 175,8160,762,coach
8,d9c16c729a1cd13eb08161a9d60b70d8,2022-04-16,2022-04-17,ATL,LGA,B0AJZSN1,PT2H19M,0,False,False,True,171.16,198.60,7,762.0,1650206400,2022-04-17T10:40:00.000-04:00,1650214740,2022-04-17T12:59:00.000-04:00,LGA,ATL,JetBlue Airways,B6,Embraer 175,8340,762,coach
9,363081f0b1bbcad5bae624e98b70d23c,2022-04-16,2022-04-17,ATL,LGA,B0AJZSN1,PT2H22M,0,False,False,True,171.16,198.60,1,762.0,1650240600,2022-04-17T20:10:00.000-04:00,1650249120,2022-04-17T22:32:00.000-04:00,LGA,ATL,JetBlue Airways,B6,Embraer 170,8520,762,coach


# Null Handling

In [22]:
print("Finding nulls per column")
df.isnull().any()

Finding nulls per column


legId                                False
searchDate                           False
flightDate                           False
startingAirport                      False
destinationAirport                   False
fareBasisCode                        False
travelDuration                       False
elapsedDays                          False
isBasicEconomy                       False
isRefundable                         False
isNonStop                            False
baseFare                             False
totalFare                            False
seatsRemaining                       False
totalTravelDistance                   True
segmentsDepartureTimeEpochSeconds    False
segmentsDepartureTimeRaw             False
segmentsArrivalTimeEpochSeconds      False
segmentsArrivalTimeRaw               False
segmentsArrivalAirportCode           False
segmentsDepartureAirportCode         False
segmentsAirlineName                  False
segmentsAirlineCode                  False
segmentsEqu

In [59]:
#looking at when `totalTravelDistance` is null
print("Where totalTravelDistance is null")
df_but_null_total_dist = df[
    df["totalTravelDistance"].isnull()
][["totalTravelDistance", "segmentsDistance"]]
display(df_but_null_total_dist.head())


#looking at when `segmentsDistance` is null
print("Where segmentsDistance is null")
df_but_null_segment_dist = df[
    df["segmentsDistance"].isnull()
][["totalTravelDistance", "segmentsDistance"]]
display(df_but_null_segment_dist.head())


#looking at when `segmentsDistanc` and `totalTravelDistance` is null
print("Where segmentsDistance and totalTravelDistance is null")
df_but_null_total_and_segment_dist = df[
    (df["segmentsDistance"].isnull()) & 
    (df["segmentsDistance"].isnull())
][["totalTravelDistance", "segmentsDistance"]]
display(df_but_null_total_and_segment_dist.head())

Where totalTravelDistance is null


,totalTravelDistance,segmentsDistance


Where segmentsDistance is null


,totalTravelDistance,segmentsDistance


Where segmentsDistance and totalTravelDistance is null


,totalTravelDistance,segmentsDistance


In [23]:
print("Observing null columns")
df[["totalTravelDistance", "segmentsEquipmentDescription", "segmentsDistance"]].head()

Observing null columns


,totalTravelDistance,segmentsEquipmentDescription,segmentsDistance
0,947.0,NaN,947
1,947.0,NaN,947
2,1675.0,||Embraer 190,947||728
3,1247.0,Embraer 190||Embraer 190,762||485
4,762.0,Embraer 190,762


In [24]:
null_col_sums = df.isna().sum()
display(null_col_sums)
null_numerators = [x for x in null_col_sums if x!= 0]

denominator = df.shape[0] #we are using the shape tuple to pull in the total rows to accurately understand how much is missing on a grand perspective

for null_numerator in null_numerators:
    print((null_numerator/denominator)*100)

legId                                    0
searchDate                               0
flightDate                               0
startingAirport                          0
destinationAirport                       0
fareBasisCode                            0
travelDuration                           0
elapsedDays                              0
isBasicEconomy                           0
isRefundable                             0
isNonStop                                0
baseFare                                 0
totalFare                                0
seatsRemaining                           0
totalTravelDistance                  56436
segmentsDepartureTimeEpochSeconds        0
segmentsDepartureTimeRaw                 0
segmentsArrivalTimeEpochSeconds          0
segmentsArrivalTimeRaw                   0
segmentsArrivalAirportCode               0
segmentsDepartureAirportCode             0
segmentsAirlineName                      0
segmentsAirlineCode                      0
segmentsEqu

0.8269689527638898
0.7684586574136486
0.2852834811354485


With the JetBlue data subset only showing around 0.82% of nulls existent (total elements...not rows), the executive decision that comes to mind is to remove the nulls. However, one thing to still do is to count the unique pairings of these elements to understand exactly what we are removing. 

In [25]:
#find a way to filter all unique pairings of T/F...
#does this warrant some feature engineering?

df["totalTravelDistance_null_existence"] =  df["totalTravelDistance"].isnull()
df["segmentsDistance_null_existence"] = df["segmentsDistance"].isnull()
df["segmentsEquipmentDescription_null_existence"] = df["segmentsEquipmentDescription"].isnull()

df.head()

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeEpochSeconds,segmentsDepartureTimeRaw,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode,totalTravelDistance_null_existence,segmentsDistance_null_existence,segmentsEquipmentDescription_null_existence
0,79eda9f841e226a1e2121d74211e595c,2022-04-16,2022-04-17,ATL,BOS,VH0AUEL1,PT2H38M,0,False,False,True,217.67,248.60,7,947.0,1650206700,2022-04-17T10:45:00.000-04:00,1650216180,2022-04-17T13:23:00.000-04:00,BOS,ATL,JetBlue Airways,B6,NaN,9480,947,coach,False,False,True
1,24c07cf38de8a7a0ed2d7709168042a8,2022-04-16,2022-04-17,ATL,BOS,QH0KUEL1,PT2H44M,0,False,False,True,316.73,355.09,7,947.0,1650233220,2022-04-17T18:07:00.000-04:00,1650243060,2022-04-17T20:51:00.000-04:00,BOS,ATL,JetBlue Airways,B6,NaN,9840,947,coach,False,False,True
2,a291a2b5ac19223c3407cfec58d1fc24,2022-04-16,2022-04-17,ATL,CLT,VH0AUEL1,PT8H48M,0,False,False,False,482.79,542.60,7,1675.0,1650206700||1650229500,2022-04-17T10:45:00.000-04:00||2022-04-17T17:0...,1650216180||1650238380,2022-04-17T13:23:00.000-04:00||2022-04-17T19:3...,BOS||CLT,ATL||BOS,JetBlue Airways||JetBlue Airways,B6||B6,||Embraer 190,9480||8880,947||728,coach||coach,False,False,False
3,d53bcf9e8b6523fa2de9420d7cb22196,2022-04-16,2022-04-17,ATL,DTW,RH0AUEL1,PT5H8M,0,False,False,False,269.78,313.61,7,1247.0,1650235800||1650246600,2022-04-17T18:50:00.000-04:00||2022-04-17T21:5...,1650244380||1650254280,2022-04-17T21:13:00.000-04:00||2022-04-17T23:5...,JFK||DTW,ATL||JFK,JetBlue Airways||JetBlue Airways,B6||B6,Embraer 190||Embraer 190,8580||7680,762||485,coach||coach,False,False,False
4,5bb138f39804320ece767e747608a50e,2022-04-16,2022-04-17,ATL,EWR,VC0QBEL1,PT2H9M,0,False,False,True,171.16,198.60,7,762.0,1650195000,2022-04-17T07:30:00.000-04:00,1650202740,2022-04-17T09:39:00.000-04:00,EWR,ATL,JetBlue Airways,B6,Embraer 190,7740,762,coach,False,False,False


In [26]:
print(df.groupby(["totalTravelDistance_null_existence", "segmentsDistance_null_existence", "segmentsEquipmentDescription_null_existence"]).size())

print(f'The length of the df null existence series is: {len(df.groupby(["totalTravelDistance_null_existence", "segmentsDistance_null_existence", "segmentsEquipmentDescription_null_existence"]).size())}')

totalTravelDistance_null_existence  segmentsDistance_null_existence  segmentsEquipmentDescription_null_existence
False                               False                            False                                          6715947
                                                                     True                                             52057
True                                False                            False                                            36967
                                    True                             False                                            19083
                                                                     True                                               386
dtype: int64
The length of the df null existence series is: 5


From the top down, we have a series of the values where we grouped the column pairings of the True/False null existence features. The length is 5 and we're going to only select relevant features here. 

In [27]:
len(df.groupby(["totalTravelDistance_null_existence", "segmentsDistance_null_existence", "segmentsEquipmentDescription_null_existence"]).size())

5

In [28]:
df.groupby(["totalTravelDistance_null_existence", "segmentsDistance_null_existence", "segmentsEquipmentDescription_null_existence"]).size().sum()

6824440

In [35]:
distinct_null_counter_array = df.groupby(["totalTravelDistance_null_existence", "segmentsDistance_null_existence", "segmentsEquipmentDescription_null_existence"]).size()

print(f"""
The nulls we are concerned with: {distinct_null_counter_array[1]} + {distinct_null_counter_array[2]} + {distinct_null_counter_array[4]}
""")

result = distinct_null_counter_array[1] + distinct_null_counter_array[2] + distinct_null_counter_array[4]

print(f"This is the total amount of loss: {result}")

print(f"The percentage of total nulls compounded by distinction is: {result / df.groupby(['totalTravelDistance_null_existence', 'segmentsDistance_null_existence', 'segmentsEquipmentDescription_null_existence']).size().sum()}")


The nulls we are concerned with: 52057 + 36967 + 386

This is the total amount of loss: 89410
The percentage of total nulls compounded by distinction is: 0.0131014412904209


/var/folders/gf/trjkr8p14xbfkbwsfn2w5_kw0000gn/T/ipykernel_2171/3343332512.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  The nulls we are concerned with: {distinct_null_counter_array[1]} + {distinct_null_counter_array[2]} + {distinct_null_counter_array[4]}
/var/folders/gf/trjkr8p14xbfkbwsfn2w5_kw0000gn/T/ipykernel_2171/3343332512.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  result = distinct_null_counter_array[1] + distinct_null_counter_array[2] + distinct_null_counter_array[4]


So we found the true data loss due to nulls and the result is 0.01% of our data to be removed (summing to a total of 89,410 rows out of our 6.8M long dataframe. Let's remove the total code recognized nulls.)

In [38]:
df.dropna(inplace=True)

In [41]:
df.shape

(6715947, 30)

# Error Checking

**Cleaning Poor Labeling (Equipment)**

- I saw some places where `"||"` exists for either segments distance or equipment. 
    - To see such, I should look at the distinct values of these dataframes. 

I saw some places where `"||"` exists for either segments distance or equipment. 

To see such, I should look at the distinct values of these dataframes. 

In [63]:
columns_to_see_classification_error_in = ["segmentsEquipmentDescription", "segmentsAirlineCode", "segmentsAirlineName", "segmentsArrivalAirportCode", "segmentsArrivalTimeEpochSeconds", "segmentsDepartureTimeEpochSeconds", "segmentsCabinCode"]

sorted(df[columns_to_see_classification_error_in[0]].unique())

['AIRBUS INDUSTRIE A321 SHARKLETS',
 'AIRBUS INDUSTRIE A321 SHARKLETS||',
 'AIRBUS INDUSTRIE A321 SHARKLETS||AIRBUS INDUSTRIE A321 SHARKLETS',
 'AIRBUS INDUSTRIE A321 SHARKLETS||Airbus A319',
 'AIRBUS INDUSTRIE A321 SHARKLETS||Airbus A319-321',
 'AIRBUS INDUSTRIE A321 SHARKLETS||Airbus A319-321||Airbus A320',
 'AIRBUS INDUSTRIE A321 SHARKLETS||Airbus A320',
 'AIRBUS INDUSTRIE A321 SHARKLETS||Airbus A321',
 'AIRBUS INDUSTRIE A321 SHARKLETS||Boeing 737 MAX 8',
 'AIRBUS INDUSTRIE A321 SHARKLETS||Boeing 737-800',
 'AIRBUS INDUSTRIE A321 SHARKLETS||Boeing 777-200',
 'AIRBUS INDUSTRIE A321 SHARKLETS||Embraer 175',
 'AIRBUS INDUSTRIE A321 SHARKLETS||Embraer 190',
 'Airbus A319',
 'Airbus A319-321',
 'Airbus A319-321||',
 'Airbus A319-321||AIRBUS INDUSTRIE A321 SHARKLETS',
 'Airbus A319-321||Airbus A319',
 'Airbus A319-321||Airbus A320',
 'Airbus A319-321||Airbus A320||',
 'Airbus A319-321||Airbus A320||Airbus A320',
 'Airbus A319-321||Airbus A320||Embraer 190',
 'Airbus A319-321||Airbus A321'

In [68]:
df.groupby(columns_to_see_classification_error_in[0]).size().sort_values(ascending=False)

segmentsEquipmentDescription
Boeing 737-800                                                                                                                    1152884
Airbus A319-321                                                                                                                    490818
Embraer 190                                                                                                                        482274
Embraer 190||Embraer 190                                                                                                           428326
Airbus A320                                                                                                                        396667
Airbus A320||Airbus A320                                                                                                           318251
Airbus A321                                                                                                                        294106
Airbu

`regex` might be necessary here to identify those areas where we notice some problematic strings. However, identifying such can be done by first filtering down to anything that contains at least `|`. 

In [77]:
df[
    df[columns_to_see_classification_error_in[0]].str.contains("|||", case=False, regex=False) 
].groupby(columns_to_see_classification_error_in[0]).size().sort_values(ascending=False) #regex is false because | is naturally treated as an "OR" command in python

segmentsEquipmentDescription
Airbus A320||||Embraer 190        585
Embraer 190||||Airbus A319-321    237
Airbus A320||||Airbus A320        202
Embraer 190||||Airbus A320        131
Airbus A320||||                    40
Embraer 190||||Embraer 190         13
Airbus A320||||Airbus A321         10
||||Airbus A320                     3
Airbus A321||||Airbus A320          1
Embraer 190||||Boeing 737-800       1
||||Embraer 190                     1
dtype: int64

In [76]:
len(df[
    df[columns_to_see_classification_error_in[0]].str.contains("|", case=False, regex=False) 
].groupby(columns_to_see_classification_error_in[0]).size().sort_values(ascending=False)) #regex is false because | is naturally treated as an "OR" command in python

255

In [79]:
df[
    df[columns_to_see_classification_error_in[0]].str.contains("|||", case=False, regex=False) 
]

,legId,searchDate,flightDate,startingAirport,destinationAirport,fareBasisCode,travelDuration,elapsedDays,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeEpochSeconds,segmentsDepartureTimeRaw,segmentsArrivalTimeEpochSeconds,segmentsArrivalTimeRaw,segmentsArrivalAirportCode,segmentsDepartureAirportCode,segmentsAirlineName,segmentsAirlineCode,segmentsEquipmentDescription,segmentsDurationInSeconds,segmentsDistance,segmentsCabinCode,totalTravelDistance_null_existence,segmentsDistance_null_existence,segmentsEquipmentDescription_null_existence
2415995,c4e9eb3627ed22dd0b8b58e2cac14681,2022-06-25,2022-08-04,PHL,LAX,PL2ABEL1,PT16H52M,0,False,False,False,346.98,406.70,7,2923.0,1659620700||1659650940||1659658500,2022-08-04T09:45:00.000-04:00||2022-08-04T18:0...,1659626400||1659655800||1659681420,2022-08-04T11:20:00.000-04:00||2022-08-04T19:3...,BOS||JFK||LAX,PHL||BOS||JFK,JetBlue Airways||JetBlue Airways||JetBlue Airways,B6||B6||B6,Embraer 190||||Airbus A319-321,5700||4860||22920,280||185||2458,coach||coach||coach,False,False,False
2415996,152cbd60711e9ec28362a6b2dd492ece,2022-06-25,2022-08-04,PHL,LAX,PL2ABEL1,PT18H8M,1,False,False,False,346.98,406.70,7,2923.0,1659620700||1659650940||1659663600,2022-08-04T09:45:00.000-04:00||2022-08-04T18:0...,1659626400||1659655800||1659685980,2022-08-04T11:20:00.000-04:00||2022-08-04T19:3...,BOS||JFK||LAX,PHL||BOS||JFK,JetBlue Airways||JetBlue Airways||JetBlue Airways,B6||B6||B6,Embraer 190||||Airbus A319-321,5700||4860||22380,280||185||2458,coach||coach||coach,False,False,False
2415997,f94ad2f45f9f92886e17e2cd941fad67,2022-06-25,2022-08-04,PHL,LAX,PL2ABEL1,PT19H14M,1,False,False,False,346.98,406.70,7,2923.0,1659620700||1659650940||1659667500,2022-08-04T09:45:00.000-04:00||2022-08-04T18:0...,1659626400||1659655800||1659689940,2022-08-04T11:20:00.000-04:00||2022-08-04T19:3...,BOS||JFK||LAX,PHL||BOS||JFK,JetBlue Airways||JetBlue Airways||JetBlue Airways,B6||B6||B6,Embraer 190||||Airbus A319-321,5700||4860||22440,280||185||2458,coach||coach||coach,False,False,False
2415999,ad8a8023b66e87971b3ccaac1396003f,2022-06-25,2022-08-04,PHL,LAX,OL4AUEL1,PT12H47M,0,False,False,False,374.88,431.10,7,2923.0,1659635400||1659650940||1659658500,2022-08-04T13:50:00.000-04:00||2022-08-04T18:0...,1659640920||1659655800||1659681420,2022-08-04T15:22:00.000-04:00||2022-08-04T19:3...,BOS||JFK||LAX,PHL||BOS||JFK,JetBlue Airways||JetBlue Airways||JetBlue Airways,B6||B6||B6,Embraer 190||||Airbus A319-321,5520||4860||22920,280||185||2458,coach||coach||coach,False,False,False
2416000,b08919497386c2d5ebada933b7980cd7,2022-06-25,2022-08-04,PHL,LAX,OL4AUEL1,PT14H3M,1,False,False,False,374.88,431.10,7,2923.0,1659635400||1659650940||1659663600,2022-08-04T13:50:00.000-04:00||2022-08-04T18:0...,1659640920||1659655800||1659685980,2022-08-04T15:22:00.000-04:00||2022-08-04T19:3...,BOS||JFK||LAX,PHL||BOS||JFK,JetBlue Airways||JetBlue Airways||JetBlue Airways,B6||B6||B6,Embraer 190||||Airbus A319-321,5520||4860||22380,280||185||2458,coach||coach||coach,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6799173,ba5e50469b87ac7d38d9b12f82071748,2022-10-05,2022-10-18,LAX,EWR,PI7QUSL1,PT14H59M,1,False,False,False,218.61,268.71,7,2805.0,1666136400||1666161000||1666184400,2022-10-18T16:40:00.000-07:00||2022-10-18T23:3...,1666140720||1666178940||1666190340,2022-10-18T17:52:00.000-07:00||2022-10-19T07:2...,LAS||BOS||EWR,LAX||LAS||BOS,JetBlue Airways||JetBlue Airways||JetBlue Airways,B6||B6||B6,Airbus A320||||Embraer 190,4320||17940||5940,241||2373||191,coach||coach||coach,False,False,False
6810554,4719a18ae3a2821b8d1b4ed9409c32f4,2022-10-05,2022-10-29,LAX,EWR,PI7QUSL1,PT15H22M,1,False,False,False,101.40,142.71,7,2805.0,1667086800||1667111400||1667136600,2022-10-29T16:40:00.000-07:00||2022-10-29T23:3...,1667091120||1667129340||1667142120,2022-10-29T17:52:00.000-07:00||2022-10-30T07:2...,LAS||BOS||EWR,

There is more error with the `segmentsEquipmentDescription`. When referencing some trips with multiple legs (seeing it displayed in the `segmentsAirlineCode` column), there are multiple leg trips. Great. However, the labeling of the equipment used to travel is incorrect. For instance, `legId` `c4e9eb3627ed22dd0b8b58e2cac14681` shows that there are 3 legs to the trip, but the equipment is separated by one delimiter of `||||`. <-- side note, this is interesting in count because it showcases enough `||` to house one more element. 

Anyway...the aircraft shown here is: `Embraer 190||||Airbus A319-321`. The Airbus A319-321 is an aircraft that does not exist. What might be implied is that there are two aircrafts being spoken of on this Airbus comment. I believe the legs correspond to two Airbus aircraft types: A319 and A321. 

How is this to be addressed and cleaned? 

I may have to drop this altogether. Or otherwise, consider removing multi-leg trips. Or consider bringing in another dataset to verify the integrity of these aircraft. Or consider create a rule that states to only look at any thing that has `||` only in length + direct routes. Or I remove anything that has a `-` in its aircraft name. This last suggesstion isn't one I would be proud of. 

**

# Data Integrity Check (outlier detection)

Make sure we aren't getting an outrageous values. 

**Unrealistic Values**

- check dates aren't outrageous
- check that mileage isn't outrageous
- check that seat numbers aren't outrageous
- 

**Outlier Detection**

# Feature Transformation

Let's address a conversion of time and maybe considering an additional feature that best represents time. S

Can we verify the seconds from one column actually equates to the dates in another. 

Do we need to clean one feature of legs into two or 3 features? Using additional features to understand important legs of trip. 

**Removing Duplicated Features**

- test to remove Airline Code and Airline Name
    - need to assert that there are a limited amount of distinct values (if so, then it means the feature is not that important as we aren't using it compare against other airlines).

In [80]:
##airline code uniqueness

sorted(df[columns_to_see_classification_error_in[1]].unique())

['9K||9K||9K||B6',
 '9K||9K||B6',
 '9K||B6',
 '9K||B6||B6',
 'AA||B6',
 'B6',
 'B6||3M||B6',
 'B6||9K',
 'B6||9K||9K',
 'B6||9K||B6',
 'B6||AA',
 'B6||B6',
 'B6||B6||9K',
 'B6||B6||B6']

In [81]:
df.groupby(columns_to_see_classification_error_in[1]).size().sort_values(ascending=False)

segmentsAirlineCode
B6                3506867
B6||B6            3092858
B6||B6||B6         108877
B6||9K||9K           2142
9K||B6               1842
B6||9K               1353
B6||9K||B6           1035
AA||B6                388
9K||9K||B6            249
B6||B6||9K            126
9K||B6||B6            116
B6||3M||B6             80
B6||AA                 13
9K||9K||9K||B6          1
dtype: int64

In [82]:
##airline

sorted(df[columns_to_see_classification_error_in[2]].unique())

['American Airlines||JetBlue Airways',
 'Cape Air||Cape Air||Cape Air||JetBlue Airways',
 'Cape Air||Cape Air||JetBlue Airways',
 'Cape Air||JetBlue Airways',
 'Cape Air||JetBlue Airways||JetBlue Airways',
 'JetBlue Airways',
 'JetBlue Airways||American Airlines',
 'JetBlue Airways||Cape Air',
 'JetBlue Airways||Cape Air||Cape Air',
 'JetBlue Airways||Cape Air||JetBlue Airways',
 'JetBlue Airways||JetBlue Airways',
 'JetBlue Airways||JetBlue Airways||Cape Air',
 'JetBlue Airways||JetBlue Airways||JetBlue Airways',
 'JetBlue Airways||Silver Airways||JetBlue Airways']

In [83]:
df.groupby(columns_to_see_classification_error_in[2]).size().sort_values(ascending=False)

segmentsAirlineName
JetBlue Airways                                      3506867
JetBlue Airways||JetBlue Airways                     3092858
JetBlue Airways||JetBlue Airways||JetBlue Airways     108877
JetBlue Airways||Cape Air||Cape Air                     2142
Cape Air||JetBlue Airways                               1842
JetBlue Airways||Cape Air                               1353
JetBlue Airways||Cape Air||JetBlue Airways              1035
American Airlines||JetBlue Airways                       388
Cape Air||Cape Air||JetBlue Airways                      249
JetBlue Airways||JetBlue Airways||Cape Air               126
Cape Air||JetBlue Airways||JetBlue Airways               116
JetBlue Airways||Silver Airways||JetBlue Airways          80
JetBlue Airways||American Airlines                        13
Cape Air||Cape Air||Cape Air||JetBlue Airways              1
dtype: int64

Additional reduction in some of these may be necessary. I propose first ridding all the rows that have a different airline labeled that isn't JetBlue. These airlines included in routes may sway price, but they complicate cleaning for a future model. Furthermore, they are less significant thatn the rest of the data which may also introduce some bias in the modeling. I'm going to count how many unique value instances comprise of only JetBlue flights and then those that are mixed. We want to remove the mixed version to make a strong JetBlue model. 

**KPI Creation**

- dollar/mile or mile/dollar
    - mile/dollar is easier to understand; dollar/mile is less intuitive as we're trying to use pennies swaying to measure vs mile swaying
- averages and medians for classification feature
- standard deviation of stuff

# Feature Reduction

Consider if `segmentsAirlineCode` is really necessary. It seems to only contain `B6` for airline codes which is a common Jet Blue indicator. 